<a href="https://colab.research.google.com/github/gcosma/DECODEclinicalTrialCalc/blob/main/SF36.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from scipy import stats
from typing import Dict, List

class PowerCalculator:
    """Power calculator implementing Methods 1 and 3 from Walters (2004) with power validation."""

    def __init__(self):
        self.alpha = 0.05  # Two-sided
        self.z_alpha = stats.norm.ppf(1 - self.alpha / 2)
        self.target_power = 0.8
        self.z_beta = stats.norm.ppf(self.target_power)

    def adjust_for_attrition(self, n: float, attrition_rate: float, attrition_timepoints: int) -> Dict:
        """Adjusts sample size before and after attrition."""
        if not (0 <= attrition_rate < 1):
            raise ValueError("Attrition rate must be between 0 and 1.")

        # Increase baseline sample size to adjust for attrition
        n_before_attrition = np.ceil(n)
        n_to_recruit = np.ceil(n_before_attrition / ((1 - attrition_rate) ** attrition_timepoints))
        n_after_attrition = np.floor(n_to_recruit * (1 - attrition_rate) ** attrition_timepoints)

        return int(n_before_attrition), int(n_to_recruit), int(n_after_attrition)

    def method1_normal(self, mean_difference: float, std_dev: float, attrition_rate: float = 0.1, attrition_timepoints: int = 1) -> Dict:
        """Method 1: Normal distribution sample size estimation."""
        if mean_difference <= 0 or std_dev <= 0:
            raise ValueError("Mean difference and standard deviation must be positive.")

        delta = mean_difference / std_dev
        n = 10
        while True:
            achieved_power = self.calculate_normal_power(n, mean_difference, std_dev)
            if achieved_power >= self.target_power:
                break
            n += 1

        # Adjust for attrition
        n_baseline, n_to_recruit, n_final = self.adjust_for_attrition(n, attrition_rate, attrition_timepoints)

        return {
            'method': 'Method 1 (Normal Distribution)',
            'mean_difference': mean_difference,
            'std_dev': std_dev,
            'attrition_rate': attrition_rate,
            'attrition_timepoints': attrition_timepoints,
            'baseline_n_per_group': n_baseline,
            'total_baseline_n': n_baseline * 2,
            'recruit_n_per_group': n_to_recruit,
            'total_recruit_n': n_to_recruit * 2,
            'final_n_per_group': n_final,
            'total_final_n': n_final * 2,
            'achieved_power': achieved_power,
            'power_sufficient': achieved_power >= self.target_power
        }

    def method3_ordinal(self, odds_ratio: float, category_props: List[float], attrition_rate: float = 0.1, attrition_timepoints: int = 1) -> Dict:
        """Method 3: Finds the smallest sample size for 80% power using Whitehead's method."""
        if odds_ratio <= 0:
            raise ValueError("Odds ratio must be positive.")
        if abs(sum(category_props) - 1) > 1e-6:
            raise ValueError("Category proportions must sum to 1.")

        sum_squared_props = sum(p**2 for p in category_props)
        numerator = 6 * (self.z_alpha + self.z_beta) ** 2
        denominator = np.pi * (1 - sum_squared_props) * (np.log(odds_ratio)) ** 2
        n = np.ceil(numerator / denominator)

        # Adjust for attrition
        n_baseline, n_to_recruit, n_final = self.adjust_for_attrition(n, attrition_rate, attrition_timepoints)

        achieved_power = self.calculate_ordinal_power(n_final, odds_ratio, category_props)

        return {
            'method': 'Method 3 (Ordinal)',
            'odds_ratio': odds_ratio,
            'category_props': category_props,
            'attrition_rate': attrition_rate,
            'attrition_timepoints': attrition_timepoints,
            'baseline_n_per_group': n_baseline,
            'total_baseline_n': n_baseline * 2,
            'recruit_n_per_group': n_to_recruit,
            'total_recruit_n': n_to_recruit * 2,
            'final_n_per_group': n_final,
            'total_final_n': n_final * 2,
            'achieved_power': achieved_power,
            'power_sufficient': achieved_power >= self.target_power
        }

    def calculate_normal_power(self, n: float, mean_difference: float, std_dev: float) -> float:
        """Calculate achieved power for normal method."""
        if n <= 0:
            return 0
        delta = mean_difference / std_dev
        z_value = delta * np.sqrt(n / 2) - self.z_alpha
        return stats.norm.cdf(z_value)

    def calculate_ordinal_power(self, n: float, odds_ratio: float, category_props: List[float]) -> float:
        """Calculate achieved power for ordinal method."""
        if n <= 0:
            return 0
        sum_squared_props = sum(p**2 for p in category_props)
        z_value = (np.log(odds_ratio) * np.sqrt(n * np.pi * (1 - sum_squared_props) / 6)) - self.z_alpha
        return stats.norm.cdf(z_value)

    def print_results(self, result: Dict) -> None:
        """Prints the results in a clear and organized format."""
        print("\n" + "=" * 50)
        print(f"          {result['method']} Results")
        print("=" * 50)

        # Summary
        print("\nSummary:")
        print(f"- Achieved Power: {result['achieved_power']:.3f}")
        print(f"- Power Sufficient: {'Yes' if result['power_sufficient'] else 'No'}")
        print(f"- Final Sample Size (per group): {result['final_n_per_group']}")
        print(f"- Total Final Sample Size: {result['total_final_n']}")

        # Input Parameters
        print("\nInput Parameters:")
        if result['method'] == 'Method 1 (Normal Distribution)':
            print(f"  - Mean Difference: {result['mean_difference']}")
            print(f"  - Standard Deviation: {result['std_dev']}")
        elif result['method'] == 'Method 3 (Ordinal)':
            print(f"  - Odds Ratio: {result['odds_ratio']}")
            print(f"  - Category Proportions: {result['category_props']}")
        print(f"  - Attrition Rate: {result['attrition_rate']}")
        print(f"  - Attrition Timepoints: {result['attrition_timepoints']}")

        # Sample Sizes
        print("\nSample Sizes:")
        print(f"  - Baseline (per group, before attrition): {result['baseline_n_per_group']}")
        print(f"  - Baseline (total, before attrition): {result['total_baseline_n']}")
        print(f"  - Initial Recruitment (per group, adjusted for attrition): {result['recruit_n_per_group']}")
        print(f"  - Initial Recruitment (total, adjusted for attrition): {result['total_recruit_n']}")
        print(f"  - Final (per group, after attrition): {result['final_n_per_group']}")
        print(f"  - Final (total, after attrition): {result['total_final_n']}")

        print("=" * 50)


# Example usage
if __name__ == "__main__":
    # Create calculator instance
    calc = PowerCalculator()

    # Example parameters
    mean_difference = 5
    std_dev = 20
    odds_ratio = 1.5
    category_props = [0.2, 0.3, 0.3, 0.2]
    attrition_rate = 0.1

    # Run methods
    results = [
        calc.method1_normal(mean_difference, std_dev, attrition_rate),
        calc.method3_ordinal(odds_ratio, category_props, attrition_rate),
    ]

    # Print results
    for result in results:
        calc.print_results(result)


          Method 1 (Normal Distribution) Results

Summary:
- Achieved Power: 0.801
- Power Sufficient: Yes
- Final Sample Size (per group): 252
- Total Final Sample Size: 504

Input Parameters:
  - Mean Difference: 5
  - Standard Deviation: 20
  - Attrition Rate: 0.1
  - Attrition Timepoints: 1

Sample Sizes:
  - Baseline (per group, before attrition): 252
  - Baseline (total, before attrition): 504
  - Initial Recruitment (per group, adjusted for attrition): 280
  - Initial Recruitment (total, adjusted for attrition): 560
  - Final (per group, after attrition): 252
  - Final (total, after attrition): 504

          Method 3 (Ordinal) Results

Summary:
- Achieved Power: 0.802
- Power Sufficient: Yes
- Final Sample Size (per group): 124
- Total Final Sample Size: 248

Input Parameters:
  - Odds Ratio: 1.5
  - Category Proportions: [0.2, 0.3, 0.3, 0.2]
  - Attrition Rate: 0.1
  - Attrition Timepoints: 1

Sample Sizes:
  - Baseline (per group, before attrition): 124
  - Baseline (total, 